In [1]:
import pandas as pd 
import numpy as np
import tensorly as tl
import plotly.express as px
import matplotlib.pyplot as plt 
from tensorly.decomposition import CP,tucker, parafac, non_negative_tucker
from datetime import timedelta, date
from statsmodels.tsa.stattools import acf, pacf

In [2]:
df_FR = pd.read_excel('FINANCIAL_RATIOS_NEW.xlsx', index_col =[0], skiprows=[1])

In [3]:
price = None
FR_dic = {}
FR_list = ['PE', 'PX_LAST', 'PS','PB']
n = 0 
NUMBER_OF_STOCKS = 220
for i,name in enumerate(FR_list) : 
    FR_dic[name] = (df_FR.iloc[:,n:n+NUMBER_OF_STOCKS]
                    .set_index(df_FR.index)
                    )

    if name not in ['PX_LAST', 'MCAP'] : 
        # FR_dic[name] = (FR_dic[name] - FR_dic[name].mean()) / FR_dic[name].std()
        FR_dic[name] = FR_dic[name].diff()
    if name in ['PX_LAST'] :

        price = FR_dic['PX_LAST']
        FR_dic[name] = np.log(FR_dic[name])
        FR_dic[name] = FR_dic[name].diff()
    

    
    FR_dic[name] = FR_dic[name][1:]
    n += NUMBER_OF_STOCKS + 2

COLUMN_NAMES = FR_dic['PE'].columns
FR_index = FR_dic['PE'].index

for key, val in FR_dic.items() : 
    FR_dic[key] = FR_dic[key][1:]



In [74]:
df_TI = pd.read_excel('TECH_NEW.xlsx', skiprows=[0])

TI_dic = {}
TI_list = ['BB_PERCENT', 'HURST', 'MOM', 'ROC','RSI','WLPR']
n = 1

for _, TI in enumerate(TI_list) : 
    filter_col = [col for col in df_TI if col.startswith(TI)]
    TI_dic[TI] = (df_TI[filter_col]
                 .set_axis(COLUMN_NAMES, axis=1)
                 .set_index(df_TI['Dates'])
                 .iloc[:-1]
                 .fillna(method = 'ffill')
                 .set_index(FR_index)
                 .diff()
                 .iloc[1:]
                 )
    # TI_dic[TI] = (TI_dic[TI] - TI_dic[TI].mean()) / TI_dic[TI].std()
TI_dic.pop('ROC', None)
TI_dic.pop('WLPR', None)
TI_dic.pop('BB_PERCENT', None)
TI_dic.pop('RSI', None)
# TI_dic.pop('HURST', None)
# TI_dic.pop('MOM', None)






,AMAT US Equity,COKE US Equity,WDFC US Equity,AAPL US Equity,KLAC US Equity,SEIC US Equity,CSPI US Equity,ALOT US Equity,AMGN US Equity,CAMP US Equity,...,DVN US Equity,ORCL US Equity,PG US Equity,INTC US Equity,TTMI US Equity,TTEK US Equity,WFC US Equity,WMT US Equity,C US Equity,BA US Equity
2006-11-24,-2.224233,-0.453817,-1.517323,4.841130,3.387437,1.096661,-1.312913,2.897439,-3.120883,-1.105011,...,0.146298,2.252991,1.893400,-4.501691,1.649787,-4.532297,-5.840585,1.202504,1.595611,3.793508
2006-12-01,-2.053757,-1.161186,2.554492,1.029270,-3.785001,-3.176736,0.854699,0.794800,-0.874210,0.311577,...,2.428829,-7.622190,-2.923864,-3.503785,-2.067527,-4.515547,-0.390235,-4.191311,-6.367649,-2.825137
2006-12-08,2.827996,3.392074,0.319400,-3.802464,1.560281,1.693252,3.191663,4.231011,-7.711691,2.237694,...,0.621117,-14.751177,2.297295,-2.359614,1.087782,1.348066,0.453615,-1.242478,5.029320,2.456348
2006-12-15,-5.463387,2.190771,0.619418,-1.499352,-5.481471,-1.666538,-7.922685,-2.253118,-0.139993,-0.104012,...,-4.614273,0.986996,-2.805150,-1.394327,-6.813869,1.650935,-1.483038,-2.269033,6.812452,-2.949547
2006-12-22,2.563554,-1.523547,5.373134,-7.671856,1.015592,1.766941,-2.424392,4.289810,-1.142916,12.671741,...,-4.660249,-8.413056,4.168915,-0.721105,-0.109521,-2.300894,4.215124,-0.108761,8.626871,0.697624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-29,-4.319681,-5.418509,-1.822813,-7.034069,-6.333237,-5.052269,-6.201765,1.361242,-6.199239,-3.127722,...,-10.947924,-5.867009,-7.145496,-4.898988,-5.522428,-6.414245,-7.175406,-6.748935,-4.132088,-8.527885
2022-05-06,7.752239,6.566924,-1.650809,6.288856,8.973904,2.436015,6.915645,2.174468,-9.821068,6.042185,...,8.255261,0.659840,-3.021952,2.938458,6.078284,0.128834,3.360934,0.316319,5.990293,1.203183
2022-05-13,-6.643218,0.363220,-2.254292,-10.396870,-6.947908,-8.384277,-4.233947,-2.582282,2.783983,-4.938289,...,-4.276851,-4.063099,-6.452394,-5.764848,-3.898360,-8.218085,-5.282535,-7.891160,-5.174845,-6.269296
2022-05-20,5.084828,-1.738245,-0.181066,-2.442297,5.049160,4.358786,7.519199,-0.412611,1.975760,9.140310,...,3.926319,-2.405455,-8.106560,-0.685424,3.508007,0.625243,-0.658935,-17.502356,3.534116,-0.758714


In [75]:
look_back_duration = 2
look_forward = 1
n_stocks = 12
n_features = 6


In [76]:
ft1 =  np.stack(FR_dic.values(), axis=2)
ft2 = np.stack(TI_dic.values(), axis=2)
# feature_tensor=ft2
feature_tensor = np.dstack((ft1, ft2))

total_pc = np.zeros(shape=[2,n_stocks,n_features])
list_of_features = [] #Split the features into different look_back_duration time slots. 

for k in range(0,len(feature_tensor),look_back_duration): 
    core, factors = tucker(feature_tensor[k:k+look_back_duration], rank= [2,n_stocks,n_features])
    pc = tl.tenalg.mode_dot(feature_tensor[k:k+look_back_duration], factors[1].T, mode = 1)
    total_pc = np.vstack((total_pc,pc))
    rec = tl.tucker_to_tensor((core,factors))
    rec_error = tl.norm(rec - feature_tensor[k:k+look_back_duration])/tl.norm(feature_tensor[k:k+look_back_duration])
    print(rec_error)
total_pc = total_pc[2:]
list_of_features = [] #Split the features into different look_back_duration time slots. 
for i in range(0,len(total_pc)-look_back_duration-look_forward+1) : 
    list_of_features.append(total_pc[i:i+look_back_duration, :,:])


c:\Users\zackx\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3364: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


1.3189271372945887e-15
7.648126856106465e-16
5.857458078350206e-16
1.6928216651164426e-15
3.9701249856705404e-16
1.223581860763359e-15
1.0420200992642226e-15
1.7477124423734366e-15
1.0016326569229115e-15
4.154641467524285e-16
8.963953887680022e-16
1.7752101749319097e-15
1.0747992758363487e-15
1.4601792930706108e-15
2.2887259500419654e-15
1.0751775404845096e-15
1.2343153703020915e-15
8.323645289694882e-16
9.40342502854075e-16
1.296311231305445e-15
1.1764840106712863e-15
1.6375067249647821e-15
1.7215296503921503e-15
8.730079999526987e-16
7.44586753769494e-16
1.0226929886767526e-15
1.6912776597298674e-15
1.2465265455864145e-15
1.3655983301238745e-15
1.8288036288786735e-15
1.924971877291855e-15
8.538866813553382e-16
7.014880631748881e-16
1.1527327400692458e-15
1.1557035190199857e-15
5.871851950584403e-16
1.3264556227968707e-15
1.1617451837048265e-15
1.6647874421555108e-15
1.539485479698381e-15
1.203628306134788e-15
2.053514607230778e-15
1.4377216145875412e-15
1.4721877164513224e-15
1.25989

In [77]:
# for i in range(7) : 
#     plt.figure()
#     for n in range(n_stocks) :
#         plt.plot(total_pc[:,n,i])

In [78]:
def get_solo_features(solo_df, lb_duration, lf_duration) :
    list_of_solo_f = []
    #SOLO FEATURES
    for i in range(0,len(solo_df)-lb_duration-lf_duration+1) : 
        list_of_solo_f.append(solo_df[i:i+lb_duration].to_numpy())
    return list_of_solo_f

### GET THE RETURNS OF COMPANY X

In [79]:
#Append the training data of test company 
solo_df = pd.DataFrame()

test_company = 'AAPL'
k = 0
for key,value in FR_dic.items() : 
    if k == 0 : 
        solo_df[key] = value[f'{test_company} US Equity']
    if k > 0 : 
        solo_df[key] = value[f'{test_company} US Equity'+ f'.{k}']
    k+=1

for key,value in TI_dic.items() : 
    solo_df[key] = value[f'{test_company} US Equity']
    

for col in FR_dic['PX_LAST'] : 
    if col.startswith(test_company) : 
        y_predict = FR_dic['PX_LAST'][col]

test_comp_returns = pd.DataFrame(y_predict).set_axis(['Log Returns'], axis = 1)
test_comp_returns['Cumulative Log Returns'] = test_comp_returns['Log Returns'].cumsum()
def get_n_week_retuns(log_returns, look_back_duration, lookforward) : 
    sdate = test_comp_returns.index.values[0]
    edate = test_comp_returns.index.values[-1]
    s = (log_returns
     .reset_index()
     .iloc[look_back_duration:]
    )
    n_week_retuns = s.rolling(lookforward).sum()
    n_week_retuns.index = list(pd.date_range(sdate ,edate + pd.to_timedelta(2, unit='D') ,freq='w') - pd.to_timedelta(2, unit='D'))[:-look_back_duration]
    return n_week_retuns.dropna() 


def to_simple_return(cum_log_ret) : 
    return np.exp(cum_log_ret) - 1


In [146]:
from numpy import newaxis
# x_ret_list contains the 
y_arr =  get_n_week_retuns(test_comp_returns['Log Returns'], look_back_duration = look_back_duration, lookforward = look_forward)

y_arr[(y_arr > 0)] = 1

y_arr[y_arr <= 0] = -1

y_ret = y_arr.to_numpy().flatten()


list_of_solo_f = get_solo_features(solo_df, look_back_duration, look_forward)
list_of_combined_features = []


for feature_t,solo_feature in zip(list_of_features,list_of_solo_f)  :
    solo_feature = solo_feature[:,newaxis,:]
    print(feature_t.shape)

    tmp = np.concatenate((feature_t,solo_feature), axis=1)
    list_of_combined_features.append(tmp)


X_Cols = [tensor.flatten() for tensor in list_of_features ]

S_Cols = [mat.flatten() for mat in list_of_solo_f]

C_Cols = [tensor.flatten() for tensor in list_of_combined_features]

C_val, Y_val = C_Cols[-120:], y_ret[-120:]

S_val = S_Cols[-120:]

X_Cols, S_Cols, C_Cols = X_Cols[:-120], S_Cols[:-120] , C_Cols[:-120]



(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)
(2, 12, 6)

In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from numpy import newaxis
 

X_train, X_test, Y_train, Y_test = train_test_split(X_Cols, y_ret[:-120], random_state = 42, shuffle=False)

S_train, S_test, Y_train, Y_test = train_test_split(S_Cols, y_ret[:-120], random_state = 42, shuffle=False)

C_train, C_test, Y_train, Y_test = train_test_split(C_Cols, y_ret[:-120], random_state = 42, shuffle=False)


In [148]:
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, mean_squared_error, classification_report



In [95]:
# ## HYPERPARAMETER TUNING
# n_estimators = [1000,2000]

# max_features = ['auto']

# max_depth = [4]

# min_samples_split = [2]

# min_samples_leaf = [2]

# bootstrap = [False, True]


# random_grid = {
    
# 'n_estimators': n_estimators,

# 'max_features': max_features,

# 'max_depth': max_depth,

# 'min_samples_split': min_samples_split,

# 'min_samples_leaf': min_samples_leaf,

# 'bootstrap' : bootstrap

# }

# rf = RandomForestClassifier()

# # Using 5 fold cross validation
# # Search across 100 different combintations and use all available cores

# rf_random = RandomizedSearchCV(
#     estimator = rf,
#     param_distributions= random_grid,
#     n_iter = 1, 
#     verbose= 4,
#     cv = 2,
#     random_state= 144,
#     n_jobs = 4,
    
# )

# rf_random.fit(C_train, Y_train)


Fitting 2 folds for each of 1 candidates, totalling 2 fits


c:\Users\zackx\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_iter=1, n_jobs=4,
                   param_distributions={'bootstrap': [False, True],
                                        'max_depth': [3],
                                        'max_features': ['auto'],
                                        'min_samples_leaf': [2],
                                        'min_samples_split': [2],
                                        'n_estimators': [1000, 2000]},
                   random_state=144, verbose=4)

In [131]:
def baseline(Y_test, prediction) : 
    Y_test = Y_test.flatten()

    all_ones = np.ones(len(Y_test))
    all_neg_ones = np.ones(len(Y_test))* -1
    print(f"Actual number of ones : {np.count_nonzero(Y_test == 1)}")
    print(f"Predicted number of ones : {np.count_nonzero(prediction == 1)}")
    print(f"Baseline if all were ones : {np.count_nonzero((all_ones==Y_test)) / len(Y_test)}")

    print(f"Actual number of -1 : {np.count_nonzero(Y_test == -1)}")
    print(f"Predicted number of -1 : {np.count_nonzero(prediction == -1)}")
    print(f"Baseline if all were -1 : {np.count_nonzero((all_neg_ones==Y_test)) / len(Y_test)}")



Actual number of ones : 69
Predicted number of ones : 107
Baseline if all were ones : 0.575
Actual number of -1 : 51
Predicted number of -1 : 13
Baseline if all were -1 : 0.425


In [156]:
rand_frst_S = RandomForestClassifier(n_estimators= 500, min_samples_split = 2, min_samples_leaf=3,  max_features='auto', max_depth = 3, bootstrap = True, random_state=22)

rand_frst_S.fit(S_train,Y_train)


prediction = rand_frst_S.predict(S_train)
print(accuracy_score(Y_train, prediction))
print(confusion_matrix(Y_train, prediction))

prediction = rand_frst_S.predict(S_test)
print(accuracy_score(Y_test, prediction))
print(confusion_matrix(Y_test, prediction))

prediction2 = rand_frst_S.predict(S_val)
print(accuracy_score(Y_val, prediction2))
print(confusion_matrix(Y_val, prediction2))

baseline(Y_test,prediction)





c:\Users\zackx\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


0.6724806201550387
[[ 56 165]
 [  4 291]]
0.5872093023255814
[[15 48]
 [23 86]]
0.5083333333333333
[[24 27]
 [32 37]]
Actual number of ones : 109
Predicted number of ones : 134
Baseline if all were ones : 0.6337209302325582
Actual number of -1 : 63
Predicted number of -1 : 38
Baseline if all were -1 : 0.36627906976744184


In [97]:
# print('Random grid: ', random_grid, '\n')

# print('Best Parameters: ', rf_random.best_params_, ' \n')

# # print(rf_random.best_score_)
# print(rf_random.best_params_)
# target_names = ['Red', 'Green']
# prediction = rf_random.predict(C_train)
# print(accuracy_score(Y_train, prediction))
# # print(classification_report(Y_train, prediction, target_names=target_names))
# print(confusion_matrix(Y_train, prediction))


# prediction = rf_random.predict(C_test)
# print(accuracy_score(Y_test, prediction))
# # print(classification_report(Y_test, prediction, target_names=target_names))
# print(confusion_matrix(Y_test, prediction))


# prediction_val1= rf_random.predict(C_val)
# print(accuracy_score(Y_val, prediction_val1))
# print(confusion_matrix(Y_val, prediction_val1))

Random grid:  {'n_estimators': [1000, 2000], 'max_features': ['auto'], 'max_depth': [3], 'min_samples_split': [2], 'min_samples_leaf': [2], 'bootstrap': [False, True]} 

Best Parameters:  {'n_estimators': 2000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 3, 'bootstrap': False}  

{'n_estimators': 2000, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 3, 'bootstrap': False}
0.7189922480620154
[[ 76 145]
 [  0 295]]
0.6046511627906976
[[ 5 58]
 [10 99]]
0.5333333333333333
[[14 37]
 [19 50]]


In [143]:
# feature_imp = pd.Series(rf_random.best_estimator_.feature_importances_)
feature_imp = pd.Series(rand_frst_C.feature_importances_)

feature_imp = feature_imp.sort_values(ascending=True)
srted_indx_list = feature_imp.index.to_list()

col_names= ['PE', 'PX_LAST', 'PS', 'PB', 'HURST', 'MOM']
# col_names = ['BB_PERCENT', 'HURST', 'MOM', 'ROC','RSI','WLPR']

def get_feature_name (srted_indx_list) :
    name_list = []
    for num in srted_indx_list : 
        half = len(srted_indx_list)/2 
        if num < half : 
            rows = num // n_features
            cols = num % n_features

            name = f'1.{col_names[cols]}.{rows}' 
            
        else : 
            rows = num // n_features - n_stocks - 1
            cols = num % n_features
            name = f'2.{col_names[cols]}.{rows}' 
    
        name_list.append(name)
    
    return name_list


feature_imp.index = (get_feature_name(srted_indx_list) )

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(feature_imp)


        #second time frame

2.PB.3          0.001304
2.PE.12         0.001553
1.PB.10         0.001912
2.PE.4          0.002009
1.PS.11         0.002021
1.PS.6          0.002119
2.PB.6          0.002441
1.PX_LAST.6     0.002627
1.MOM.3         0.002654
2.PE.11         0.002725
2.HURST.6       0.002760
1.MOM.5         0.002814
1.PS.5          0.002876
2.PX_LAST.3     0.002897
1.PS.10         0.002973
2.PE.8          0.003006
2.PX_LAST.12    0.003062
1.PX_LAST.11    0.003073
1.PB.5          0.003086
2.MOM.7         0.003110
1.HURST.11      0.003127
2.PX_LAST.6     0.003133
2.HURST.0       0.003172
2.PB.12         0.003177
1.PX_LAST.7     0.003188
2.PB.4          0.003190
1.HURST.6       0.003288
1.PE.9          0.003340
2.PS.10         0.003350
1.PB.9          0.003424
1.MOM.9         0.003444
2.PB.1          0.003468
2.MOM.9         0.003676
2.PS.6          0.003703
2.PS.12         0.003720
1.PE.10         0.003753
2.PS.4          0.003774
2.MOM.2         0.003821
2.HURST.11      0.003843
1.HURST.5       0.003881


In [87]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(feature_imp.sort_values(ascending=False))

2.BB_PERCENT.5     0.047663
1.WLPR.1           0.037198
1.WLPR.10          0.036964
1.MOM.1            0.033582
1.BB_PERCENT.3     0.029337
1.ROC.1            0.028627
2.MOM.11           0.025825
1.ROC.2            0.025127
1.BB_PERCENT.1     0.024101
1.ROC.8            0.021638
1.WLPR.2           0.020580
2.BB_PERCENT.0     0.018286
1.RSI.9            0.017695
1.MOM.12           0.016158
1.HURST.3          0.015845
1.BB_PERCENT.0     0.014275
1.WLPR.0           0.013896
2.RSI.9            0.012093
1.ROC.3            0.011656
2.ROC.5            0.011215
2.HURST.8          0.011055
1.MOM.3            0.010858
1.WLPR.4           0.010455
1.RSI.1            0.010283
2.BB_PERCENT.7     0.010170
2.RSI.12           0.010145
2.RSI.3            0.009812
1.WLPR.6           0.009641
1.RSI.10           0.009613
2.BB_PERCENT.2     0.009348
2.ROC.0            0.008905
1.ROC.6            0.008773
2.WLPR.12          0.008502
2.ROC.2            0.008192
1.MOM.8            0.008186
1.WLPR.12          0

In [88]:



# baseline(Y)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
# # FR_dic['PE']['AAPL US Equity'].rolling(50).mean().plot()
# # FR_dic['PE']['SLB US Equity'].rolling(50).mean().plot()
# aapl_acf = acf(FR_dic['PE']['AAPL US Equity'])
# aapl_acf_2 = acf(FR_dic['PX_LAST']['AAPL US Equity.1'])
# aapl_acf_3 = acf(FR_dic['PS']['AAPL US Equity.2'])
# aapl_acf_4 = acf(FR_dic['PB']['AAPL US Equity.3'])
# aapl_acf_5 = acf(TI_dic['HURST']['AAPL US Equity'])
# aapl_acf_6 = acf(TI_dic['HURST']['AAPL US Equity'])

# # aapl_acf_6 = acf(FR_dic['PB']['AAPL US Equity.3'])


# test_df = pd.DataFrame([aapl_acf, aapl_acf_2, aapl_acf_3, aapl_acf_4, aapl_acf_5, aapl_acf_6]).T
# test_df.columns = ['ACF of first-order difference of PE of AAPL', 'ACF of first-order difference of PX_LAST of AAPL', 
#                    'ACF of first-order difference of PS of AAPL', 'ACF of first-order difference of PB of AAPL', 'ACF of first-order difference of HURST of AAPL',
#                    'ACF of first-order difference of Momentum of AAPL'
#                   ] 
# test_df.index += 1
# test_df.plot(kind='bar', figsize=(10,7))

# plt.savefig('ACF.png')